## Bank Term Deposit Acceptance forecasting

Obiettivo di questo assignment è la valutazione delle competenze tecniche del candidato e la sua modalità di approccio ad un problema di Data Science. Nello specifico è richiesto di sviluppare un modello predittivo in grado di indicare se un cliente intercettato da una campagna di marketing da parte di una banca decide di sottoscrivere o meno un deposito bancario a termine (bank term deposit).

#### Dataset: ####

All'interno della cartella **data**  viene fornito il file **bank-dataset.csv** che contiene le campagne marketing telefoniche effettuate da una banca per proporre l'acquisto del prodotto bancario.
I dettagli del dataset sono forniti all'interno del file: **bank-names.txt**.
La variabile target che indica se il cliente accetta o meno la sottoscrizione del deposito bancario è contenuta nel medesimo file con field name "y".

#### Assignement: ####

Richiesta di questo assignment è la costruzione di un modello predittivo con performance soddisfacenti per il candidato dando evidenza di tutti gli step tipici che dovrebbero essere affrontati in un progetto di Data Science: dalla pulizia e preparazione del dato fino al testing delle performance del modello costruito.

Il notebook svolto dovrà essere opportunamente commentato e dovrà essere consegnato tramite condivisione di un repository github personale accessibile che ne permetta la riproduzione.

### Import library and custom funcitons

In [ ]:
import numpy as np
import pandas as pd 
import seaborn as sns
import matplotlib.pyplot as plt

%matplotlib inline 

from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix

In [ ]:
def check_values(df, col, **kwargs):
    """
    Function than wrap pandas value_counts functions in order to check multiple columns
    :param df: input dataframe
    :param col: selected column
    :param **kwargs: pandas value_counts params
    """
    
    print(f"\nColumn: {col}\n")
    print(df[col].value_counts(**kwargs))
    print("#"*100)
    
def normalize_col(df, col, nan_value="unknown"):
    """
    Apply lower case and replace nan value to a column
    :param df: input dataframe
    :param col: selected column
    :param nan_value: value to replace in nan (default: 'unknown')
    :return dataframe
    """
    
    df[col] = df[col].str.lower()
    df[col] = df[col].replace(np.nan, 0)

    return df

def replace_missing(df):
    """
    Check and replace missing value with mode
    :param df: input dataframe
    :return dataframe
    """
        
    for col in df.columns:
        isna = df[col].isna().values.any()
        if isna:
            new_value = df[col].mode()[0]
            df[col] = df[col].fillna(new_value)
            print(f"Replaced {col} missing value with nan values: {new_value}")   
        
    return df

def plot_correlation_matrix(df, single_corner=False, target=None):
    """
    Plot correlation matrix with option to convert target column and plot half matrix 
    :param df: input dataframe
    :param single_corner: boolean to plot only half matrix (default: False)
    :param target: name of target column to convert into numerical column (default: None)
    """
    
    df_corr = df.copy()
    if target:
        binary_map = {"no": 0, "yes": 1}
        df_corr[target] = df_corr[target].map(binary_map)
        
    df_corr = df_corr.corr()    
    if single_corner:
        mask_corr = np.zeros_like(df_corr, dtype=bool)
        mask_corr[np.triu_indices_from(mask_corr)] = True
        df_corr[mask_corr] = np.nan
        
    sns.set(rc={'figure.figsize':(10,8)}, style='darkgrid')   
    corr_matrix = sns.heatmap(df_corr, vmin=0, vmax=1, annot =True, cmap="YlGnBu")
    plt.show()

In [ ]:
def plot_bivariate_analysis(df, col, target="y"):
    """
    Plot bivariate histogram with col values and target value
    :param df: input dataframe
    :param col: selected column
    :param target : name of target column (default: 'y')
    """
    
    sns.set(rc={'figure.figsize':(14,10)}, style='darkgrid')
    
    colors = ["coral", "lightgreen"]
    sns.set_palette(sns.color_palette(colors))
    
    plt_col = sns.countplot(x=col, data = df, hue = target, order = df[col].value_counts().index)
    
    plt_col.tick_params(axis='x', rotation=50)
    plt.title(f"Relationship between {col} and {target}",fontsize=18)
    plt.show()
    

    
def plot_numerical_col(df,col,target='y'):
    """
    Plot bivariate boxplot and histogram with col values and target value with mean ,mode, median values
    :param df: input dataframe
    :param col: selected column
    :param target : name of target column (default: 'y')
    """
    
    mean=df[col].mean()
    median=df[col].median()
    mode=df[col].mode().values[0]
    
    f, (box, hist) = plt.subplots(2, sharex=True, gridspec_kw= {"height_ratios": (0.3, 1)})
    sns.set(rc={'figure.figsize':(14,10)}, style='darkgrid')
    
    colors = ["coral", "lightgreen"]
    sns.set_palette(sns.color_palette(colors))
    
    sns.boxplot(data=df, x=col, y=target, ax=box, order = df[target].value_counts().index)
    sns.histplot(data=df, x=col, kde=True, ax=hist)
    
    box.axvline(mean, color='r')
    box.axvline(median, color='g')
    box.axvline(mode, color='b')
    box.set(xlabel='')
    box.set_title(f"Relationship between {col} and {target}",fontsize=18)

    hist.axvline(mean, color='r', label="Mean")
    hist.axvline(median, color='g', label="Median")
    hist.axvline(mode, color='b', label="Mode")
    hist.legend()
    
    plt.show()
    print("\n\n")
    
def plot_confusion_matrix(y_pred, y_true):
    """
    Plot confusion matrix
    :param y_pred: predicted values
    :param y_true: true value
    """
    cm = confusion_matrix(y_pred, y_true)
    sns.set(rc={'figure.figsize':(8,6)}, style='darkgrid')   
    sns.heatmap(cm, annot=True,fmt='g',cmap="YlGnBu")
    plt.show()

## EDA (Exploraty data analysis)

In [ ]:
df = pd.read_csv("./data/bank-dataset.csv")

In [ ]:
df.head()

### Check values for all columns

In [ ]:
obj_cols = list(df.select_dtypes(include="object").columns)
num_cols = list(df.select_dtypes(exclude="object").columns)

##### Object values

La variabile target è molto sbilanciata sulla classe no-> 93% vs 7%

In [ ]:
for col in obj_cols:
    check_values(df,col,normalize=True)

#### Normalize string column with lowercase and replace NaN

In [ ]:
for col in obj_cols:
    df = normalize_col(df,col)

#### Fix wrong values for marital values

In [ ]:
mask_single = df["marital"].str.startswith("s")
mask_divorced = df["marital"].str.startswith("d")

df.loc[mask_divorced, "marital"] = "divorced"
df.loc[mask_single, "marital"] = "single"

##### Check null/nan values 

Le colonne **age, duration** hanno dei valori mancanti. I valori mancanti verrà sostituiti con le relative mode.

In [ ]:
df = replace_missing(df)

### Correlation

Non ci sono features correlate fra di loro e neanche con la variabile target

In [ ]:
plot_correlation_matrix(df, single_corner=True, target="y")

### Plot bivariate analysis

In [ ]:
for col in num_cols:
    plot_numerical_col(df,col)

In [ ]:
for col in obj_cols:
    plot_bivariate_analysis(df,col)

### Features selection

Dai precendenti plot risulta opportuno eliminare le seguenti features: **month, day, pdays, previous**

#### Drop features

In [ ]:
cols_to_drop = ["month", "day", "pdays", "previous"]
df = df.drop(columns=cols_to_drop)

#### Convert binary cols

In [ ]:
binary_cols = ["default","housing","loan","y"]
binary_map = {"no": 0, "yes": 1}

for col in binary_cols:
    df[col] = df[col].map(binary_map)
    obj_cols.remove(col)

#### Convert categorical cols (OneHotEncoding)

In [ ]:
categorical_cols = ['job', 'marital', 'contact', 'education', 'poutcome']
df = pd.get_dummies(df, columns=categorical_cols)

### Oversampling with SMOTE

In [ ]:
y = df["y"]
X = df.drop("y",axis = 1)

sm = SMOTE()
X_sm , y_sm = sm.fit_resample(X, y)

### Split train/test

In [ ]:
X_train , X_test , y_train , y_test = train_test_split(X_sm, y_sm, test_size = 0.2, random_state = 123456)

### Train model 
Effettuo il training con una GridSearchCV per fare il tuning di alcuni hyperparametri utilizzando la CrossValidation

In [ ]:
param_grid = {
    'max_depth': [None, 50],
    "max_features" : ['auto', 'sqrt']
}

In [ ]:
model = RandomForestClassifier()
grid_search = GridSearchCV(estimator = model, param_grid = param_grid, cv = 3, n_jobs = -1, verbose = 2)
grid_search.fit(X_train,y_train)

#### Best parameters

In [ ]:
grid_search.best_params_

#### Best model

In [ ]:
best_model = grid_search.best_estimator_

### Predict result on test set

In [ ]:
y_pred = best_model.predict(X_test)

### Metriche

In [ ]:
plot_confusion_matrix(y_test, y_pred)

In [ ]:
print(classification_report(y_test, y_pred))

Tutte le metriche hanno dei valori alti, dunque il modello addestrato riesce a classificare correttamente entrambe le classi